In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import plotly.express as px 

In [2]:
scan_1 = pd.read_csv("nap-scan.csv")
scan_2 = pd.read_csv("nap-scan-2.csv")
cols = ["ApId",
"NApMacAddress",
"LastCheckinTime",
"LastStatusId",         
"SSID",                 
"TSF",                  
"Frequency",            
"Rssi",                  
"PrimaryChannel",       
"SecondaryChannel",     
"ChannelWidth",         
"StaChannelWidth",      
"FirstCenterFrequency", 
"SecondCenterFrequency",
"Authentication",       
"Security"]
scan_1.columns = cols
# checking the column 
print(scan_2[r'\N'].value_counts(), len(scan_2[r'\N']))
# seems like the \N col is having only newline values dropping it 
scan_2 = scan_2.drop(r"\N", axis = 1 ) 
scan_2.columns = cols
total_scan = scan_1.append(scan_2, ignore_index=True)

\N    5259
Name: \N, dtype: int64 5259


/tmp/ipykernel_12851/3795742374.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_scan = scan_1.append(scan_2, ignore_index=True)


In [3]:
total_scan 

,ApId,NApMacAddress,LastCheckinTime,LastStatusId,SSID,TSF,Frequency,Rssi,PrimaryChannel,SecondaryChannel,ChannelWidth,StaChannelWidth,FirstCenterFrequency,SecondCenterFrequency,Authentication,Security
0,297,EA:48:B8:C2:6F:BC,2022-04-05 04:46:18,1275036,x20Free_Wi-Fi,3806851836,11,-59,11,0,NaN,20,NaN,NaN,NaN,NaN
1,320,04:95:E6:9E:80:78,2022-04-07 10:12:19,1298899,L.S.C,43136848531,10,-72,10,0,NaN,20,NaN,NaN,PSK,CCMP
2,320,04:95:E6:A6:50:90,2022-04-07 10:12:19,1298899,Singla,45709517200,2,-83,2,0,NaN,any,NaN,NaN,PSK,CCMP
3,320,32:84:2F:EB:A5:40,2022-04-07 10:12:19,1298899,Redmi 9A,1276727306001,11,-85,11,0,0.0,20,0.0,0.0,PSK,CCMP
4,320,34:CE:00:23:5D:00,2022-04-07 10:12:19,1298899,Xiaomi_5CFF,12711629212,1,-84,1,0,NaN,20,NaN,NaN,PSK,TKIP CCMP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6198,2187,72:6D:EC:0A:4C:23,2022-04-11 11:39:33,15711825,NaN,369069911408,153,-55,127,0,0.0,20,153.0,0.0,PSK,CCMP TKIP
6199,2187,72:6D:EC:0A:83:33,2022-04-11 11:39:33,15711825,INDIO-cast-failover,525933864222,149,-51,127,0,1.0,any,155.0,0.0,PSK,CCMP TKIP
6200,2187,72:6D:EC:0A:8A:17,2022-04-11 11:39:33,15711825,INDIO-cast-failover,5466269661876,52,-49,52,0,1.0,any,58.0,0.0,PSK,CCMP TKIP
6201,2187,76:6D:EC:0A:4C:23,2022-04-11 11:39:33,15711825,NaN,369069945541,153,-55,127,0,0.0,20,153.0,0.0,PSK,CCMP TKIP


In [4]:
total_scan.isna().sum()

ApId                        0
NApMacAddress               0
LastCheckinTime             0
LastStatusId                0
SSID                      330
TSF                         0
Frequency                   0
Rssi                        0
PrimaryChannel              0
SecondaryChannel            0
ChannelWidth             4998
StaChannelWidth           185
FirstCenterFrequency     4998
SecondCenterFrequency    4998
Authentication           1774
Security                 1774
dtype: int64

In [5]:
cont_features = ["Rssi","ChannelWidth",]# StaChannelWidth,FirstCenterFrequency
to_predict = ["PrimaryChannel"]


In [6]:
total_scan.PrimaryChannel.value_counts()

11     1604
1      1545
6       735
127     488
10      218
0       185
9       156
36      154
2       131
52      131
8       127
7       110
4       103
3        98
5        83
13       73
40       69
44       65
48       34
60       24
64       18
116      11
12       11
100      10
56        9
108       9
104       1
124       1
Name: PrimaryChannel, dtype: int64

In [7]:
total_scan.dtypes

ApId                       int64
NApMacAddress             object
LastCheckinTime           object
LastStatusId               int64
SSID                      object
TSF                        int64
Frequency                  int64
Rssi                       int64
PrimaryChannel             int64
SecondaryChannel           int64
ChannelWidth             float64
StaChannelWidth           object
FirstCenterFrequency     float64
SecondCenterFrequency    float64
Authentication            object
Security                  object
dtype: object

In [9]:
from pycaret.classification import *

# creaing setup 
expt = setup(total_scan,target="PrimaryChannel",numeric_features =cont_features, preprocess = True ,  normalize = True , silent = True, 
use_gpu=True 
)

,Description,Value
0,session_id,2704
1,Target,PrimaryChannel
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(6203, 16)"
5,Missing Values,1
6,Numeric Features,7
7,Categorical Features,7
8,Ordinal Features,0
9,High Cardinality Features,0


In [10]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,0.9979,0.0000,0.9860,0.9977,0.9977,0.9976,0.9976,0.2790
catboost,CatBoost Classifier,0.9919,0.0000,0.8711,0.9862,0.9887,0.9905,0.9905,80.7760
gbc,Gradient Boosting Classifier,0.9880,0.0000,0.8355,0.9832,0.9847,0.9859,0.9859,279.7590
rf,Random Forest Classifier,0.8717,0.0000,0.6316,0.8625,0.8598,0.8469,0.8478,3.2510
nb,Naive Bayes,0.8551,0.0000,0.7831,0.8923,0.8614,0.8308,0.8355,0.6530
et,Extra Trees Classifier,0.8441,0.0000,0.6518,0.8364,0.8326,0.8137,0.8147,4.6340
svm,SVM - Linear Kernel,0.8091,0.0000,0.5727,0.8112,0.7951,0.7711,0.7736,4.7820
lr,Logistic Regression,0.8063,0.0000,0.5186,0.7931,0.7774,0.7645,0.7690,67.3860
ridge,Ridge Classifier,0.7685,0.0000,0.5482,0.7715,0.7509,0.7194,0.7231,1.0670
lightgbm,Light Gradient Boosting Machine,0.7615,0.0000,0.6872,0.7592,0.7474,0.7124,0.7119,2.5510


In [11]:
model = create_model(best)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,0.9977,0.0000,0.9600,0.9955,0.9966,0.9973,0.9973
2,0.9954,0.0000,0.9933,0.9961,0.9954,0.9946,0.9946
3,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,0.9977,0.0000,0.9965,0.9979,0.9977,0.9973,0.9973
5,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,0.9954,0.0000,0.9200,0.9939,0.9942,0.9946,0.9946
8,0.9977,0.0000,0.9965,0.9979,0.9977,0.9973,0.9973


In [16]:
# saving compared models 
save_model(best, "compare_model_res")

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=False, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=['Rssi',
                                                           'ChannelWidth'],
                                       target='PrimaryChannel',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_valu...
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                  

In [15]:
save_model(model , "dt")

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=False, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=['Rssi',
                                                           'ChannelWidth'],
                                       target='PrimaryChannel',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_valu...
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                  

In [12]:
def create_graph(node, df):
    node_data = df.query(f"ApId == {node}")
    print(node_data)

In [36]:
total_scan.query("ApId == 297")

,ApId,NApMacAddress,LastCheckinTime,LastStatusId,SSID,TSF,Frequency,Rssi,PrimaryChannel,SecondaryChannel,ChannelWidth,StaChannelWidth,FirstCenterFrequency,SecondCenterFrequency,Authentication,Security
0,297,EA:48:B8:C2:6F:BC,2022-04-05 04:46:18,1275036,x20Free_Wi-Fi,3806851836,11,-59,11,0,NaN,20,NaN,NaN,NaN,NaN


In [38]:
total_scan.ApId.value_counts()

678     71
719     63
967     51
2138    48
1565    47
        ..
1397     1
1444     1
1454     1
864      1
297      1
Name: ApId, Length: 422, dtype: int64

In [39]:
create_graph(1565, total_scan)

      ApId      NApMacAddress      LastCheckinTime  LastStatusId  \
3751  1565  00:95:69:E0:41:D2  2022-04-09 04:22:33      15587861   
3752  1565  00:A0:0A:24:1F:D0  2022-04-09 04:22:33      15587861   
3753  1565  00:A0:0A:27:52:50  2022-04-09 04:22:33      15587861   
3754  1565  00:A0:0A:2A:72:C0  2022-04-09 04:22:33      15587861   
3755  1565  0C:D2:B5:4B:B0:42  2022-04-09 04:22:33      15587861   
3756  1565  20:DB:AB:37:8D:80  2022-04-09 04:22:33      15587861   
3757  1565  20:DB:AB:37:8D:81  2022-04-09 04:22:33      15587861   
3758  1565  20:DB:AB:37:8D:E0  2022-04-09 04:22:33      15587861   
3759  1565  20:DB:AB:37:8D:E1  2022-04-09 04:22:33      15587861   
3760  1565  20:DB:AB:37:8D:EE  2022-04-09 04:22:33      15587861   
3761  1565  20:DB:AB:37:8D:EF  2022-04-09 04:22:33      15587861   
3762  1565  20:DB:AB:37:8E:10  2022-04-09 04:22:33      15587861   
3763  1565  20:DB:AB:37:8E:11  2022-04-09 04:22:33      15587861   
3764  1565  20:DB:AB:37:8E:1E  2022-04-09 04:22:

## Tasks to do: 
1. Create the graph showing all the nodes, color it with the channels they are connecting 
2. Try to take input into the batches i.e all the channels which the network can see according to these batches make sure that the network is running in the optimum condition for all the nodes with the distance. 